In [43]:
import sqlite3
import pandas as pd
import numpy as np

In [44]:
f_df = "./optuna_study_itr_whole_fix_lchs.db"

con = sqlite3.connect(f_df)

In [45]:
import datetime
import pickle
t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(JST)
ts = now.strftime('%Y%m%d_%H%M%S_%f')
ts

'20260104_151006_050264'

In [46]:
def d2p(d, f):
    with open(f, 'wb') as file:
        pickle.dump(d, file)

def p2d(f):
    with open(f, 'rb') as file:
        d = pickle.load(file)
    return d

def d2j(d, f):
    with open(f, "wt") as file:
        json.dump(d, file, indent=2)

def j2d(f):
    with open(f, "rt") as file:
        d = json.load(file)
    return d

In [47]:
def q2df(q):
    df = pd.DataFrame(con.execute(q))
    
    len_col = len(df.columns)
    df.columns = ["c"+str(i+1) for i in range(len_col)]
    return df

In [48]:
df_t = q2df("select name from sqlite_master where type='table';")

In [49]:
df_t

,c1
0,studies
1,version_info
2,study_directions
3,study_user_attributes
4,study_system_attributes
5,trials
6,trial_user_attributes
7,trial_system_attributes
8,trial_params
9,trial_values


In [50]:
df_s = q2df("select * from studies;")
df_s

,c1,c2
0,1,nsp-study_dummy4create
1,2,nsp-study_deg
2,3,nsp-study_deg_2
3,4,nsp-study_deg_2m
4,5,nsp-study_deg_2c
5,6,nsp-study_deg_2ch
6,7,nsp-study_deg_2i
7,8,nsp-study_deg_2f
8,9,nsp-study_deg_2f4
9,10,nsp-study_deg_2f8


In [51]:
df_ts = q2df("select * from trials;")\
    .query("c3 == 2")
df_ts.head()

,c1,c2,c3,c4,c5,c6
0,1,0,2,RUNNING,2023-01-23 17:26:10.862248,None
1,2,1,2,RUNNING,2023-01-23 17:26:10.987247,None
2,3,2,2,RUNNING,2023-01-23 17:26:10.987247,None
3,4,3,2,RUNNING,2023-01-23 17:39:24.017385,None
4,5,4,2,RUNNING,2023-01-23 17:39:24.135385,None


In [52]:
d2_tags = df_ts["c2"]

In [53]:
df_trial = q2df("select * from trial_values")\
    .query("c2 in @d2_tags")\
    .query("c4 == 0")
df_trial


,c1,c2,c3,c4,c5
1328,1329,1341,0,0.0,FINITE
1567,1568,1597,0,0.0,FINITE
1931,1932,1979,0,0.0,FINITE
1961,1962,2016,0,0.0,FINITE
2003,2004,2063,0,0.0,FINITE
2054,2055,2115,0,0.0,FINITE


In [54]:
tirals_one = df_trial["c2"]

In [55]:
df_param = q2df("select * from trial_params") \
    .query("c2 in @tirals_one")

# .query()
df_param.head()

,c1,c2,c3,c4,c5
24116,24117,1341,1by1,148.0,"{""name"": ""IntDistribution"", ""attributes"": {""lo..."
24117,24118,1341,cnight,93.0,"{""name"": ""IntDistribution"", ""attributes"": {""lo..."
24118,24119,1341,cwork,34.0,"{""name"": ""IntDistribution"", ""attributes"": {""lo..."
24119,24120,1341,dab,91.0,"{""name"": ""IntDistribution"", ""attributes"": {""lo..."
24120,24121,1341,fseq,188.0,"{""name"": ""IntDistribution"", ""attributes"": {""lo..."


In [56]:
df_param_sub = df_param \
    .loc[:, ["c2", "c3", "c4"]]
# d_params.head().to_dict()
d_params = dict()
for r in df_param_sub.itertuples():
    if r.c2 not in d_params.keys():
        d_params[r.c2] = dict()
    d_params[r.c2][r.c3] = r.c4

In [57]:
df_params_zero = pd.DataFrame().from_dict(d_params, orient="columns")\
    .assign(
            # mns = lambda df:np.mean(df, axis=1),
            med = lambda df:np.median(df, axis=1)
           )\
    .assign(
        # mns = lambda df:df.mns.astype(int),
            med = lambda df:df.med.astype(int)
        )\
    .T\
    .reset_index(drop=True).T
df_params_zero

,0,1,2,3,4,5,6
1by1,148.0,152.0,169.0,168.0,175.0,165.0,166.0
cnight,93.0,98.0,100.0,99.0,106.0,85.0,98.0
cwork,34.0,13.0,19.0,17.0,28.0,19.0,19.0
dab,91.0,85.0,105.0,115.0,114.0,116.0,109.0
fseq,188.0,227.0,227.0,226.0,229.0,222.0,226.0
lcnn,140.0,81.0,56.0,9.0,23.0,9.0,39.0
lcnt,105.0,116.0,122.0,88.0,85.0,93.0,99.0
lfbd,98.0,139.0,170.0,172.0,168.0,153.0,160.0
lnd,197.0,203.0,151.0,136.0,132.0,132.0,143.0
mps,125.0,153.0,117.0,130.0,130.0,119.0,127.0


In [58]:
d_param = df_params_zero[6].to_dict()
d_param

{'1by1': 166.0,
 'cnight': 98.0,
 'cwork': 19.0,
 'dab': 109.0,
 'fseq': 226.0,
 'lcnn': 39.0,
 'lcnt': 99.0,
 'lfbd': 160.0,
 'lnd': 143.0,
 'mps': 127.0,
 'nnv': 107.0,
 'nseq': 167.0,
 'rqh': 31.0,
 'rqn': 197.0,
 'sc': 85.0,
 'tn': 1.0,
 'tt': 11.0,
 'sweeps': 819.0}

In [59]:
int(23/3)

7

In [60]:
import importlib

import pandas as pd
import optuna
import gc
import copy

import sys
sys.path.append("../nlp_assist/")

import pandas as pd
import optuna
import gc
import copy

#条件もとの3テーブルを作るモジュール
import nlp_connect as nl
importlib.reload(nl)

#各種条件をquboとして作るためのモジュール
import nlp_limiter as lmt
importlib.reload(lmt)

#並列で探索するソルバのラッパー
import p_solver as ps
importlib.reload(ps)

#結果チェッカ等
import res_parser as rps
importlib.reload(rps)

import nsp_settings as ns
importlib.reload(ns)

print("reloaded!")

import datetime

import pickle


dummy var generator activated
start status is 0
limter activated!
reloaded!


In [61]:
dt_now = datetime.datetime.now()
dt_now.strftime('%Y%m%d_%H%M%S_%S')

'20260104_151006_06'

In [62]:
# f = "../demos/dmk0.3_20220111_21.xlsx"
# f2 = "../demos/dmk0.3_20220111_21add.xlsx"
# f = "../demos/dmk0.4_20220111_20_add202011_v2.xlsx"
f = "../demos/dmk0.4_20220111_20_add202011_v3_slim.xlsx"
f2 = "../demos/dmk0.4_20220111_20_add202011_add.xlsx"

# f = "../demos/dmk0.3_20220111_18.xlsx"
# f2 = "../demos/dmk0.3_20220111_18add.xlsx"
# x = nl.xlsx2condtion(f)
# x.make3CDT()

# # q_genはqubo generator
# qg = lmt.q_gen(x2c=x)

# nst = ns.nsp_settings()
# inclist = nst.check_list
# inclist

In [63]:
ns1 = ns.nsp_settings.param_dict.copy()
set(ns1.keys())

{'1by1',
 'cnight',
 'cwork',
 'dab',
 'eqg2',
 'eqg3',
 'fseq',
 'lcnn',
 'lcnt',
 'lfbd',
 'lnd',
 'mps',
 'nnv',
 'nseq',
 'rqh',
 'rqn',
 'sc',
 'tn',
 'tt'}

In [64]:

opt_dict = dict()
# opt_dict[0] = set(ns1.keys())
opt_dict[0] = ('1by1','cnight',
     'cwork',
     'dab',
    #  'eqg2',
    #  'eqg3',
     'fseq',
     'lcnn',
     'lcnt',
     'lfbd',
     'lnd',
     'mps',
     'nnv',
     'nseq',
     'rqh',
     'rqn',
     'sc',
     'tn',
     'tt')
print(len(opt_dict[0]))
# opt_dict[1] = ()
# opt_dict[2] = ()
# opt_dict[3] = ( 'eqg3', 'eqg2')


17


In [65]:
# d1 = ns.nsp_settings.param_dict.copy()
# d1
# from multiprocessing import Manager
# from multiprocessing import Pool
# from collections import namedtuple

In [66]:
opt_dict[0]

('1by1',
 'cnight',
 'cwork',
 'dab',
 'fseq',
 'lcnn',
 'lcnt',
 'lfbd',
 'lnd',
 'mps',
 'nnv',
 'nseq',
 'rqh',
 'rqn',
 'sc',
 'tn',
 'tt')

In [67]:
v_l = []
v_l = v_l + list(opt_dict[0])
print(v_l)

inc = v_l
d1 = ns.nsp_settings.param_dict.copy()
for t in inc:
    d1[t] = 1
    
x = nl.xlsx2condtion(f)
x.make3CDT(year=2020, month=6)

# q_genはqubo generator
qg = lmt.q_gen(x2c=x)
#qg2  = copy.deepcopy(qg)
qg.setParam(param_dict=d1)
# for i in inc:
#     print(i)
#     qg.makeAllLimitation(do_normalize=False, hideProgressBar=True, include=[i])
#     qdf = qg.ne.getQ_asDF()
#     print(qdf.head())

qg.makeAllLimitation(do_normalize=False, hideProgressBar=True, include=inc)
qdf = qg.ne.getQ_asDF()


['1by1', 'cnight', 'cwork', 'dab', 'fseq', 'lcnn', 'lcnt', 'lfbd', 'lnd', 'mps', 'nnv', 'nseq', 'rqh', 'rqn', 'sc', 'tn', 'tt']
xlsx file is
../demos/dmk0.4_20220111_20_add202011_v3_slim.xlsx
make condition tables.
holidaymode: asHoliday
year:  2020
month:  6
holidaymode:asHoliday
days that is holiday and not sunday will be Holiday.
Q generator activated!
x2c need to pre make 3DT
nsp_expression activated!
Q dict is resetted!
one can do one task per day...
limit continuous work by dt1 and member capacity
limit continious night work by dt3...
forbiden sequence is forbiden..
must pair must nor...
limitting as req numbers
limitting as req work times
limit total tasks...
limit tatal nights...
last month info: duty class to forbiden seq...
last month info: duty class to need seq...
last month info: cont work...
last month info: cont night work...
shift count quantified...
limit member pair shift...
Veteran in each group should be preserverd.
Night time novis should not be two in group.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 574229/574229 [00:00<00:00, 4668513.90it/s]


In [68]:
vs = set(qdf["n2"].to_list() + qdf["n1"].to_list())
len(set(vs))

8201

In [69]:
l_dvs = [v for v in qdf["n2"].to_list() + qdf["n1"].to_list() if "dv_" in v]
len(set(l_dvs))

3347

In [70]:
qdf.head()

,n1,n2,val
0,1_1_1_1_1,1_1_1_2_4,13
1,1_1_1_1_1,1_1_2_4_2,15
2,1_1_1_1_1,1_1_3_6_2,7
3,1_1_1_1_1,1_1_4_8_2,11
4,1_1_1_2_4,1_1_2_4_2,15


In [71]:
qdf["val"].max()

1024

In [72]:
qdf["val"].min()

-512

In [73]:
qdf \
    .assign(valint = lambda df:df.val.astype(int))\
    .query("valint != val")

,n1,n2,val,valint


In [74]:
import joblib
from joblib import Parallel, delayed
from time import time

In [75]:
10 * 11

110

In [76]:
len(qg.ne.nodes)

11548

In [77]:
import random

In [78]:
random.randint(1, 1000000)

640392

In [79]:
21/3

7.0

In [80]:
22 % 3

1

In [81]:
res_dict = dict()

In [82]:
%%time

iter_num = 10
for k in range(iter_num):
    print(k)
    print(d_param)
    swmul = d_param["sweeps"]
    param_dict = d_param.copy()
    # sw_base = param_dict["sweeps"]
    del param_dict["sweeps"]
    
    # sw_base = 1e+3
    sw_base = 1e+4
    sw = int(sw_base *swmul )
    x = nl.xlsx2condtion(f)
    x.make3CDT(year=2020, month=6)

    # q_genはqubo generator
    qg = lmt.q_gen(x2c=x)
    qg.setParam(param_dict)
    print(param_dict)
    qg.makeAllLimitation(do_normalize=True, hideProgressBar=True, include=inc)

    p = ps.psol(q=qg.ne.getQ(),
                 num_reads=1,
                 sweeps=sw,
                 seed=k)
    mins = p.p_solve_getMin(threads=18, times=18)

    rp = rps.res_parser(x=x)
    rp.prepRes(res=mins.sample, hideProgressBar=True)
    rc = rps.res_checker(x=x, echo_bool=True)
    rc.checkAllLimitation(res_parser=rp, include=inc)
    colp_num = len(rc.log_df)
    tm = rps.table_maker(x=x, res=mins.sample)


    res_dict_k = dict()
    res_dict_k["errornum"] = len(rc.log_df)
    res_dict_k["mins_sample"] = mins.sample
    res_dict_k["log_df"] = rc.log_df
    res_dict_k["timetable"]= tm.datetable.copy()
    res_dict_k["shifttable"] = tm.shifttable.copy()

    print("trial tag:" + str(k))
    print("error num:" + str(res_dict_k["errornum"]))
    print("colappse:" +str(colp_num))

    f_pkl = "./res_dict_sample_e{}_k{}_ts{}.pkl".format(res_dict_k["errornum"], k, ts)
    print(f_pkl)

    d2p(d=res_dict_k, f=f_pkl)
    
    break

0
{'1by1': 166.0, 'cnight': 98.0, 'cwork': 19.0, 'dab': 109.0, 'fseq': 226.0, 'lcnn': 39.0, 'lcnt': 99.0, 'lfbd': 160.0, 'lnd': 143.0, 'mps': 127.0, 'nnv': 107.0, 'nseq': 167.0, 'rqh': 31.0, 'rqn': 197.0, 'sc': 85.0, 'tn': 1.0, 'tt': 11.0, 'sweeps': 819.0}
xlsx file is
../demos/dmk0.4_20220111_20_add202011_v3_slim.xlsx
make condition tables.
holidaymode: asHoliday
year:  2020
month:  6
holidaymode:asHoliday
days that is holiday and not sunday will be Holiday.
Q generator activated!
x2c need to pre make 3DT
nsp_expression activated!
{'1by1': 166.0, 'cnight': 98.0, 'cwork': 19.0, 'dab': 109.0, 'fseq': 226.0, 'lcnn': 39.0, 'lcnt': 99.0, 'lfbd': 160.0, 'lnd': 143.0, 'mps': 127.0, 'nnv': 107.0, 'nseq': 167.0, 'rqh': 31.0, 'rqn': 197.0, 'sc': 85.0, 'tn': 1.0, 'tt': 11.0}
Q dict is resetted!
one can do one task per day...
limit continuous work by dt1 and member capacity
limit continious night work by dt3...
forbiden sequence is forbiden..
must pair must nor...
limitting as req numbers
limitti

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:01<00:00, 21.43it/s]


checking continenous nights works...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:01<00:00, 19.46it/s]


checking is there forbiden seq
clear!
checking needed pair is pair...
clear
number is correct?
clear
is total hour correct?
clear
is total work times is correct?
clear
is total night is correct?
clear
is forbid seq is avoided?
ﾆB Yy
NN Yy
Yy ﾆB
Yy NN
clear
is needed seq is preserved?
MY Yy
clear
checking work count from last month...
clear
checking night work count from last month
clear
checking shift count satisfication
clear
checking forbidden pair in shifts
clear
checking daytime veteran groups
clear
checking night time novice groups
clear
table_maker activated.
parser activated!
checking...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8201/8201 [00:00<00:00, 2727361.81it/s]
D:\anal\nsp_kh\note05_lmt_debug\../nlp_assist\res_parser.py:1578: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  t3_wid = t3_lim.pivot_table(values=['duty_str'], index=['date_number', "num_days"], columns=['name'],aggfunc=lambda x: ' '.join(x)).T\
D:\anal\nsp_kh\note05_lmt_debug\../nlp_assist\res_parser.py:1597: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  dt3x = pd.merge(df_Final_sub,
D:\anal\nsp_kh\note05_lmt_debug\../nlp_assist\res_parser.py:1597: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  dt3x = pd.merge(df_Final_sub,
100%|██████████████████████████████████████████████

trial tag:0
error num:0
colappse:0
./res_dict_sample_e0_k0_ts20260104_151006_050264.pkl
Wall time: 1h 35min 18s


D:\anal\nsp_kh\note05_lmt_debug\../nlp_assist\res_parser.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt1.loc[:, "weekdays"] = dt1["num_days"].copy().apply(
D:\anal\nsp_kh\note05_lmt_debug\../nlp_assist\res_parser.py:1688: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt1.loc[:, "holiday_count"] = dt1["isHoliday"].copy().apply(
D:\anal\nsp_kh\note05_lmt_debug\../nlp_assist\res_parser.py:1692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [83]:
# f_pkl = "./res_dict_sample_{}.pkl".format(ts)
# f_pkl

In [84]:
kokomade


NameError: name 'kokomade' is not defined

In [ ]:
1

In [ ]:
1